# Spam & Ham

In [277]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [278]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from tensorflow.keras.preprocessing.text import Tokenizer

In [279]:
data = pd.read_csv('./data/spam_datasets.csv', encoding='latin1')
data.dropna(axis=1, inplace=True)
data.columns = ['label', 'mail']
data.head()

,label,mail
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [280]:
# ham = 1, spam = 0
data['label'] = data['label'].replace(['ham','spam'],[0,1])
data.head()

,label,mail
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [281]:
data.mail = data.mail.str.replace("[^\w]", " ")

data.mail = data.mail.replace("", np.nan)
data.label = data.label.replace("", np.nan)
data.dropna(how='any', inplace=True)

In [282]:
data.head()

,label,mail
0,0,Go until jurong point crazy Available only ...
1,0,Ok lar Joking wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor U c already then say
4,0,Nah I don t think he goes to usf he lives aro...


In [283]:
mail_train, mail_test, y_train, y_test = train_test_split(data['mail'], data['label'], random_state=123)

In [284]:
# 불용어
stop_words = set(stopwords.words('english')) 

In [285]:
X_train = []

for words in mail_train:
    for word in words.split():
        if word not in stop_words:
            X_train.append(word)

In [286]:
X_test = []

for words in mail_test:
    for word in words.split():
        if word not in stop_words:
            X_test.append(word)

In [287]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

print(len(tokenizer.word_index))

# 단어의 빈도수가 1 이상인 것의 개수 확인
low_cnt = 0
for w, cnt in tokenizer.word_counts.items():
    if cnt > 1:
        low_cnt += 1
print(low_cnt)


7420
3609


In [288]:
# X_train 단어들을 토대로 정수 인덱스 설정, 전체 단어 갯수 설정
tokenizer = Tokenizer(4000)
tokenizer.fit_on_texts(X_train)

# 설정된 정수 인덱스를 토대로 변환
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

print('# int_encoding done')

# int_encoding done


In [289]:
# mail의 최대 길이와 평균 길이를 구해서 어느 정도의 길이로 패딩을 할지 정함


In [290]:
max_len = 50

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)


model = Sequential()
model.add(Embedding(4000, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=4)

ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, <class 'pandas.core.series.Series'>